<h1> Downscaling Landcover data using Cloud ML </h1>

In this notebook, we train a TensorFlow model to fit Landsat 8 bands to a low-resolution landcover map. Then, we use that model on the high-resolution Landsat data to create a high-resolution landcover map. In essence, we are using TensorFlow to "statistically downscale" the landcover data (note that the term "downscaling" is counterintuitive -- downscaling an image increases its resolution or upsamples it).

As a first step, we install a Python package capable of reading GDAL files.

In [2]:
# Note: after running this cell, you need to Reset Session in Datalab to pick up the new package
# You may need to change this to "!sudo apt-get" if you get permission problems.
!apt-get -y install python-gdal

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following extra packages will be installed:
  libarmadillo4 libarpack2 libdap11 libdapclient3 libdapserver7 libepsilon1
  libfreexl1 libgdal1h libgeos-3.4.2 libgeos-c1 libgif4 libhdf4-0-alt
  libhdf5-8 libjasper1 libjbig0 libjpeg62-turbo libkml0 liblcms2-2 libltdl7
  libmysqlclient18 libnetcdfc7 libodbc1 libogdi3.2 libopenjpeg5 libpoppler46
  libpq5 libproj0 libspatialite5 libtiff5 liburiparser1 libwebp5
  libxerces-c3.1 mysql-common odbcinst odbcinst1debian2 poppler-data proj-bin
  proj-data python-numpy
Suggested packages:
  libhdf4-doc libhdf4-alt-dev hdf4-tools libnetcdf4 libjasper-runtime
  liblcms2-utils libmyodbc odbc-postgresql tdsodbc unixodbc-bin ogdi-bin
  poppler-utils ghostscript fonts-japanese-mincho fonts-ipafont-mincho
  fonts-japanese-gothic fonts-ipafont-gothic fonts-arphic-ukai
  fonts-arphic-uming fonts-nanum python-nose python-numpy-dbg python-numpy-doc
The following

In [ ]:
%bash
if [ ! -d input_data ]; then
   mkdir input_data
   for band in `seq 1 7`; do
     gsutil cp gs://mdh-test/landsat-ml/landsat8-b$band.tif input_data
   done
   gsutil cp gs://mdh-test/landsat-ml/srtm-elevation.tif input_data
   gsutil cp gs://mdh-test/landsat-ml/mcd12-labels.tif input_data
fi